<a href="https://colab.research.google.com/github/bforoura/Transformers/blob/main/nlp_transformers_ch8_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title A Positional Encoding Example

# A Python library for representing documents as semantic vectors
!pip install gensim==3.8.3

# A machine learning library and a scientific computing framework
import torch

# A suite of libraries and programs for statistical language processing
import nltk

# A Sentence Tokenizer that divides a text into a list of sentences by using 
# an unsupervised algorithm
nltk.download('punkt')



In [2]:
#@title Necessary imports

import math
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize 
import gensim 
from gensim.models import Word2Vec 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings(action = 'ignore') 



In [3]:
#@title Upload the data file and parse it

dprint=0 # prints outputs if set to 1, default=0

#‘text.txt’ will reside locally in your Google colab file system 
!curl -L https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/main/Chapter08/text.txt --output "text.txt"

sample = open("text.txt", "r") 
s = sample.read() 

# processing escape characters 
f = s.replace("\n", " ") 

data = [] 

# sentence parsing 
for i in sent_tokenize(f): 
	temp = [] 
	# tokenize the sentence into words 
	for j in word_tokenize(i): 
		temp.append(j.lower()) 
	data.append(temp) 
 



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.9M  100 10.9M    0     0  11.0M      0 --:--:-- --:--:-- --:--:-- 11.0M


In [5]:
#@title Create the Skip Gram model

model = gensim.models.Word2Vec(data, min_count = 1, size = 512, window = 5, sg = 1)

print(model)


Word2Vec(vocab=11806, size=512, alpha=0.025)


In [6]:
#@title Cosine similarity function

# Cosine similarity will only be calculated if cosine==True, which means that 
# both word1 and word2 are known

def similarity(word1,word2):
        cosine=False         #default value

        try:
                a=model[word1]
                cosine=True

        except KeyError:     
                print(word1, ":[unk] key not found in dictionary") 

        try:
                b=model[word2] #a=True implied

        except KeyError:       
                cosine=False   #both a and b must be true
                print(word2, ":[unk] key not found in dictionary")

        if (cosine==True):
                b=model[word2]
                # compute cosine similarity
                dot = np.dot(a, b)
                norma = np.linalg.norm(a)
                normb = np.linalg.norm(b)
                cos = dot / (norma * normb)
                
                aa = a.reshape(1,512)
                ba = b.reshape(1,512)
     
                cos_lib = cosine_similarity(aa, ba)

         
        if(cosine==False):cos_lib=0;
        return cos_lib


In [10]:
print(similarity('freedom', 'freedom'))


[[0.9999999]]


In [11]:
#@title Case 0: Words in the dataset and the dictionary

word1="freedom"; word2="liberty"

print("Similarity", similarity(word1,word2), word1, word2)



Similarity [[0.36993998]] freedom liberty


In [13]:
#@title Case 1: Words not in the dataset or the dictionary

word1="corporations"; word2="rights"

print("Similarity", similarity(word1,word2), word1, word2)



corporations :[unk] key not found in dictionary
Similarity 0 corporations rights


In [15]:
#@title Case 2: Noisy Relationship 

word1="etext"; word2="declaration"
print("Similarity", similarity(word1,word2), word1, word2)


Similarity [[0.5619307]] etext declaration


In [16]:
#@title Case 3: Rare words

word1="justiciar"; word2="judgement"
print("Similarity", similarity(word1, word2) ,word1, word2)



Similarity [[0.24089369]] justiciar judgement


In [18]:
#@title Case 4: Replacing rare words

word1="judge"; word2="justiciar"
print("Similarity", similarity(word1,word2) ,word1, word2)



Similarity [[0.3778212]] judge justiciar


In [19]:
#@title Case 5: Entailment
word1="pay"; word2="debt"

print("Similarity", similarity(word1,word2), word1, word2)


Similarity [[0.52915555]] pay debt
